In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import os
from utils import *

jour = 102

Pour commencer on va essayer de comprendre la tendance pour pouvoir avoir une idée sur les prévisions
Ici Nous avons deux pistes la première est le graph qui donne une idée sur les prévisions et le donnes dont on dispose pour le jours ou on est actuellement (28) lors de cette edition.

# Forcasting

<img src="data/prevision_globale_edited.png" width=800>

In [2]:
T0_length_pixels = 90 
pixels_to_day = T0_length_pixels / 15
print(pixels_to_day)

T0_length = (T0_length_pixels) # px
T1_length = (T0_length + 280) # px
T2_length = (T1_length + 295) # px
T3_length = (T2_length + 155) # px 

T0_by_days = T0_length // pixels_to_day
T1_by_days = T1_length // pixels_to_day
T2_by_days = T2_length // pixels_to_day
T3_by_days = T3_length // pixels_to_day

print(f"Days with markers are {int(T0_by_days)} ,{int(T1_by_days)}, {int(T2_by_days)}, {int(T3_by_days)}")

6.0
Days with markers are 15 ,61, 110, 136


Par la suite on supposera une demande croissante en utilisera la methode du weighted moving average avec un coefficient de sécurité de 1.1 pour soit une sur estimation de la demande 10% pour l'instant vaux mieux être sur préparer que ne pas satisfaire la demande surtout dans la phase de croissance ;

In [3]:
df = pd.read_csv(f'data/demande/Demande-{jour}.csv', delimiter=';')
tmp_df = moving_average_growth_rate(df, window=3, column='Demand')
tmp_df = weighted_moving_average_rate(tmp_df, window=4, column='Demand', security_rate=1.1)

In [4]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=tmp_df['Day'], y=tmp_df['Demand'], name='Demand', line=dict(color='royalblue', width=2)))
fig.add_trace(go.Scatter(x=tmp_df['Day'], y=tmp_df['MA'], name='MA', line=dict(color='firebrick', width=2)))
fig.add_trace(go.Scatter(x=tmp_df['Day'], y=tmp_df['WMA'], name='WMA', line=dict(color='green', width=2)))

number_of_days_to_forcast = 2
forcast_next_n_days = [tmp_df.iloc[-1]['Demand']] + forcast_n_nexth_days(tmp_df, number_of_days_to_forcast, 'MA')
current_max_dayx = tmp_df['Day'].max()
days = [current_max_dayx + i for i in range(0, number_of_days_to_forcast + 1)]

fig.add_trace(go.Scatter(x=days, y=forcast_next_n_days, name='Forcast', line=dict(color='indigo', width=2)))

fig.update_layout(template='plotly_white', title='Demande-42', xaxis_title='Day', yaxis_title='Demand')
fig.show()

## Gestion de stock approximationn avec wilson

In [5]:
# forcast_next_n_days = np.array(forcast_next_n_days)

window_size = 8
forcast_next_n_days = tmp_df.iloc[-window_size:]["Demand"].to_numpy()
print(forcast_next_n_days)
mean_quantity = forcast_next_n_days.mean()
print(f"Mean quantity for next {number_of_days_to_forcast} days is {mean_quantity}")

unit_price = 150
total_quantity = forcast_next_n_days.sum() * 10
total_cost_in_time_window = total_quantity * unit_price

print(total_cost_in_time_window)

[38 32 34 38 38 35 32 43]
Mean quantity for next 2 days is 36.25
435000


In [6]:
number_of_observations = 15
data_orders = []
for product in producs_info:
    economique_quantity, total_cost = calculate_economique_size_based_on_n_observation(tmp_df, product['unit_price'], n_observation=number_of_observations)
    data_orders.append([product['name'], economique_quantity, total_cost])
    

orders_stock = pd.DataFrame(data_orders, columns=['Product', 'Economique Quantity', 'Total Cost'])
orders_stock_adopted = orders_stock.copy()[["Product", "Economique Quantity"]]
orders_stock_adopted['Economique Quantity'] = orders_stock_adopted['Economique Quantity'].apply(lambda x: np.ceil((x)/500) * 500)
orders_stock_adopted

,Product,Economique Quantity
0,Barre de silicium,3500.0
1,kit laminage,4500.0
2,Kit électronique,5500.0
3,Structure,8500.0


cherchons le point de commande pour chaque produit

In [7]:
stock_data = pd.read_csv('data/stock/Stock.csv', delimiter=';')
px.line(stock_data, x='Day', y='Level', color="Stock", title='Stock', template="plotly_white", markers=True)

In [8]:
print(f"point de commande à partir des données du jour {jour}")
for indx, product in enumerate(producs_info):
    stock_data_mat_indx = stock_data[stock_data['Stock'] == f"Matière première {indx+1}"][-2:]["Level"].to_numpy()
    delta = - stock_data_mat_indx[1] + stock_data_mat_indx[0]
    delta = delta if delta > 0 else 0
    product["security_stock"] = np.max([product["security_stock"], (product["delai"]+5)*delta])
    print(product["name"], ":", product["security_stock"])


point de commande à partir des données du jour 102
Barre de silicium : 1620
kit laminage : 1620
Kit électronique : 2000
Structure : 2000


In [30]:
production = [484, 505, 521]

In [31]:
250 * producs_info[0]["unit_price"]

37500